## Classifier using Grid Partition

#### Import libs

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import defaultdict

#### Load data and prepare

In [18]:
iris = load_iris()
X = iris.data[:, [2, 3]]
y = iris.target
target_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

#### Define Mfs

In [21]:
def get_membership_functions(data):
    min_val, max_val = np.min(data), np.max(data)
    mid_val = (min_val + max_val) / 2
    
    mfs = {
        'small': lambda x: np.maximum(0, 1 - np.abs(x - min_val) / (mid_val - min_val)),
        'medium': lambda x: np.maximum(0, 1 - np.abs(x - mid_val) / (max_val - mid_val)),
        'large': lambda x: np.maximum(0, 1 - np.abs(x - max_val) / (max_val - mid_val)),
    }
    return mfs

petal_length_mfs = get_membership_functions(X_train[:, 0])
petal_width_mfs = get_membership_functions(X_train[:, 1])
mf_names = ['small', 'medium', 'large']

#### Generate Rules using confidence

For every possible input combination (e.g., "length is small AND width is medium"), it decides what the output (the THEN part) should be.

In [22]:
def generate_rule_base(X, y, pl_mfs, pw_mfs):
    rule_base = {}
    
    for pl_name in mf_names:
        for pw_name in mf_names:
            rule_antecedent = (pl_name, pw_name)
            
            confidence_per_class = defaultdict(float)
            total_activation_strength = 0.0

            for cls in np.unique(y):
                X_class = X[y == cls]
                
                pl_activation = pl_mfs[pl_name](X_class[:, 0])
                pw_activation = pw_mfs[pw_name](X_class[:, 1])
                activation_strength = np.minimum(pl_activation, pw_activation)
                
                sum_activation_class = np.sum(activation_strength)
                confidence_per_class[cls] = sum_activation_class
            
            pl_activation_all = pl_mfs[pl_name](X[:, 0])
            pw_activation_all = pw_mfs[pw_name](X[:, 1])
            total_activation_strength = np.sum(np.minimum(pl_activation_all, pw_activation_all))
            
            if total_activation_strength > 0:
                for cls in confidence_per_class:
                    confidence_per_class[cls] /= total_activation_strength
            
            if confidence_per_class:
                best_class = max(confidence_per_class, key=confidence_per_class.get)
                rule_base[rule_antecedent] = best_class
                
    return rule_base

In [36]:
rule_base = generate_rule_base(X_train, y_train, petal_length_mfs, petal_width_mfs)

print("Extracted Rules:")
for rule, consequent in rule_base.items():
    print(f"IF petal_length is {rule[0]} AND petal_width is {rule[1]} THEN class is {target_names[consequent]}")

Extracted Rules:
IF petal_length is small AND petal_width is small THEN class is setosa
IF petal_length is small AND petal_width is medium THEN class is setosa
IF petal_length is small AND petal_width is large THEN class is versicolor
IF petal_length is medium AND petal_width is small THEN class is setosa
IF petal_length is medium AND petal_width is medium THEN class is versicolor
IF petal_length is medium AND petal_width is large THEN class is virginica
IF petal_length is large AND petal_width is small THEN class is versicolor
IF petal_length is large AND petal_width is medium THEN class is virginica
IF petal_length is large AND petal_width is large THEN class is virginica


#### Predict and evaluation

In [31]:
def predict(X_data, rules, pl_mfs, pw_mfs):
    predictions = []
    for sample in X_data:
        pl_sample, pw_sample = sample[0], sample[1]
        
        best_rule_activation = -1
        best_rule_class = -1
        
        for antecedent, consequent_class in rules.items():
            pl_name, pw_name = antecedent
            
            pl_act = pl_mfs[pl_name](pl_sample)
            pw_act = pw_mfs[pw_name](pw_sample)
            activation = min(pl_act, pw_act)
            
            if activation > best_rule_activation:
                best_rule_activation = activation
                best_rule_class = consequent_class
        
        predictions.append(best_rule_class)
        
    return np.array(predictions)

In [34]:
y_pred = predict(X_test, rule_base, petal_length_mfs, petal_width_mfs)

accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 88.89%
